In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
#Twitter API Connection
import tweepy
#NLP Library
import nltk

In [2]:
consumerKey = "xPfZgxhA6RcRGVd4Ukx2QGxn2"
consumerSecret = "qJHLTMXYweHuD3royMx157Ew7SX3n2FBFIeaA92a6EorGikELb"
accessToken = "1427378156-m6V63pdoobv5J3YYrcLPGjetQIhHZc2UeAnB0tl"
accessTokenSecret = "NIBAsbGQk516HgexRhQlVUaCmJHB8maozVbjhnqTwlBGU"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [43]:
tweets = tweepy.Cursor(api.search, q = "Biden", lang = "en", geocode = "53.8182212,-3.0564845,235km").items(3000)

In [44]:
keyword = "Biden"

In [ ]:
tweet_list=[]
loc_list=[]
date_list=[]
name=[]
followers=[]
desc=[]

for i in tweets:
  print(i._json)
  tweet_list.append(i._json["text"])
  loc_list.append(i._json["user"]["location"])
  date_list.append(datetime.datetime.strptime(i._json["created_at"], '%a %b %d %H:%M:%S +0000 %Y').strftime('%d/%m/%Y'))
  name.append(i._json["user"]["name"])
  followers.append(i._json["user"]["followers_count"])
  desc.append(i._json["user"]["description"])
  

In [46]:
tweet_dict = {"date": date_list, "name":name, "tweet":tweet_list, "description":desc, "followers":followers, "location":loc_list}

In [ ]:
tdf = pd.DataFrame(tweet_dict)
tdf

In [ ]:
nltk.download([
...     "names",
...     "stopwords",
...     "state_union",
...     "twitter_samples",
...     "movie_reviews",
...     "averaged_perceptron_tagger",
...     "vader_lexicon",
...     "punkt",
... ])

In [50]:
stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
twtstr=str(tweet_list)
print(twtstr)


In [ ]:
x=nltk.word_tokenize(twtstr)
print(x)


In [ ]:
xp=[]
for i in x:
  if i.isalpha() ==True:
    xp.append(i)
  else:
    pass

xp


In [ ]:
type(xp)

In [56]:
xp_nostop = [w for w in xp if w.lower() not in stopwords]

In [57]:
fd = nltk.FreqDist(xp_nostop)

In [ ]:
fd.tabulate(10)

In [ ]:
lower_fd = nltk.FreqDist([w.lower() for w in fd])
lower_fd

In [ ]:
text = nltk.Text(xp)
text.concordance("Biden", lines=5)

In [61]:
from nltk.sentiment import SentimentIntensityAnalyzer 
sia = SentimentIntensityAnalyzer()

In [ ]:
for i in range(0,len(tweet_list)):
  print(sia.polarity_scores(tweet_list[i]))

In [63]:
def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

In [ ]:
from random import shuffle
shuffle(tweet_list)
for t in tweet_list:
    print(">", is_positive(t), t)

In [ ]:
count_list=[]
shuffle(tweet_list)
for t in tweet_list:
    count_list.append(is_positive(t))

"{pos} POSITIVE Sentiment & , {neg} NEGATIVE Sentiment tweets on a search by keyword {key}" .format(pos=count_list.count(True), neg=count_list.count(False), key=keyword.upper())
